In [1]:
import os
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

/home/cpschau/miniforge3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


In [2]:

def plot_generation_tech(n, carrier, regions_file, title_suffix, ax, column, cmap):
    gen_tech = n.generators.query(f"carrier == '{carrier}'")
    gen_tech["carrier_type"] = gen_tech.carrier.str.extract(r"(\D+)")
    gen_tech["flh"] = n.snapshot_weightings.generators @ n.generators_t.p_max_pu[gen_tech.index]
    gen_tech.set_index("bus", inplace=True)
    regions_onshore = gpd.read_file(regions_file).set_index("shape_id")
    regions_onshore.index = regions_onshore.index + "_AC"
    regions_onshore["flh"] = gen_tech["flh"]
    regions_onshore["p_nom_max"] = gen_tech.p_nom_max

    regions_onshore.boundary.plot(ax=ax, color="black", linewidth=0.5)
    regions_onshore.plot(column=column, legend=True, ax=ax, cmap=cmap)
    ax.set_title(f"{carrier.capitalize()} - {column.replace('_', ' ').capitalize()}\n({title_suffix})", fontsize=10)
    ax.axis("off")

def plot_combined_tech(n1, n2, carrier, regions_file, output_dir):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    cmap = {"onwind": "Blues", "solar": "Reds"}[carrier]

    plot_generation_tech(n1, carrier, regions_file, "pre-overriding", axes[0, 0], "flh", cmap)
    plot_generation_tech(n2, carrier, regions_file, "post-overriding", axes[0, 1], "flh", cmap)
    plot_generation_tech(n1, carrier, regions_file, "pre-overriding", axes[1, 0], "p_nom_max", cmap)
    plot_generation_tech(n2, carrier, regions_file, "post-overriding", axes[1, 1], "p_nom_max", cmap)

    plt.tight_layout()
    output_path = os.path.join(output_dir, f"{carrier}_combined.png")
    plt.savefig(output_path)
    plt.close()

# Example usage
network_files = [
    ("/home/cpschau/Code/dev/BRIGHT/submodules/pypsa-earth-sec/pypsa-earth/networks/elec_s_11_ec_lv2.0_Co2L.nc", "pre-overriding"),
    ("/home/cpschau/Code/dev/BRIGHT/submodules/pypsa-earth-sec/results/0909_integrate_enertile/prenetworks/elec_s_11_ec_lv2.0_Co2L_144H_2030_0.071_AB_presec.nc", "post-overriding")
]
regions_file = "/home/cpschau/Code/dev/BRIGHT/submodules/pypsa-earth-sec/pypsa-earth/resources/bus_regions/regions_onshore.geojson"
carriers = ["solar", "onwind"]
presec_carriers = ["onwind2", "onwind3"]

# Create output directory if it doesn't exist
output_dir = os.path.join(os.getcwd(), "outputs")
os.makedirs(output_dir, exist_ok=True)

# Load networks
n1 = pypsa.Network(network_files[0][0])
n2 = pypsa.Network(network_files[1][0])

# Plot combined figures for solar and onwind
for carrier in carriers:
    plot_combined_tech(n1, n2, carrier, regions_file, output_dir)

# Plot individual figures for onwind2 and onwind3
for carrier in presec_carriers:
    fig, ax = plt.subplots(figsize=(10, 5))
    plot_generation_tech(n2, carrier, regions_file, "post-overriding", ax, "flh", "Blues")
    plt.tight_layout()
    output_path = os.path.join(output_dir, f"{carrier}_post-overriding_flh.png")
    plt.savefig(output_path)
    plt.close()

    fig, ax = plt.subplots(figsize=(10, 5))
    plot_generation_tech(n2, carrier, regions_file, "post-overriding", ax, "p_nom_max", "Blues")
    plt.tight_layout()
    output_path = os.path.join(output_dir, f"{carrier}_post-overriding_p_nom_max.png")
    plt.savefig(output_path)
    plt.close()

/home/cpschau/miniforge3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
/home/cpschau/miniforge3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
/home/cpschau/miniforge3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Va